In [1]:
!pip install --upgrade comtradeapicall

In [2]:
import pandas as pd
import requests
import comtradeapicall
import numpy as np
from itertools import count
import json
import math

In [3]:
from IPython.display import clear_output
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
subscription_key ='f6f38dac65574cab8a6b972e0c653a8a' # trial key, for bulk key, you will need to shop for premium

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
f = open('/content/drive/MyDrive/Trade project data/HS.json',encoding='utf-8')
hs_data = json.load(f)

In [8]:
path2 = '/content/drive/MyDrive/Trade project data/country_codes_V202401b.csv'
country_codes = pd.read_csv(path2)
codes = country_codes['country_code'].to_list()
print(codes)

[4, 8, 12, 16, 20, 24, 28, 31, 32, 36, 40, 44, 48, 50, 51, 52, 56, 58, 60, 64, 68, 70, 72, 76, 84, 86, 90, 92, 96, 100, 104, 108, 112, 116, 120, 124, 132, 136, 140, 144, 148, 152, 156, 162, 166, 170, 174, 175, 178, 180, 184, 188, 191, 192, 196, 200, 203, 204, 208, 212, 214, 218, 222, 226, 231, 232, 233, 238, 242, 246, 251, 258, 260, 262, 266, 268, 270, 275, 276, 278, 280, 288, 292, 296, 300, 304, 308, 316, 320, 324, 328, 332, 340, 344, 348, 352, 360, 364, 368, 372, 376, 380, 384, 388, 392, 398, 400, 404, 408, 410, 414, 417, 418, 422, 426, 428, 430, 434, 440, 442, 446, 450, 454, 458, 462, 466, 470, 478, 480, 484, 490, 496, 498, 499, 500, 504, 508, 512, 516, 520, 524, 528, 530, 531, 533, 534, 535, 540, 548, 554, 558, 562, 566, 570, 574, 579, 580, 583, 584, 585, 586, 591, 598, 600, 604, 608, 612, 616, 620, 624, 626, 634, 642, 643, 646, 652, 654, 659, 660, 662, 666, 670, 674, 678, 682, 686, 688, 690, 694, 697, 699, 702, 703, 704, 705, 706, 710, 711, 716, 724, 728, 729, 736, 740, 748, 752, 

In [9]:
def split_numbers(input_string, chunk_size):
    # Split the string by commas to get a list of numbers
    numbers = input_string.split(',')

    # Group the numbers into chunks of specified size
    chunks = [numbers[i:i + chunk_size] for i in range(0, len(numbers), chunk_size)]

    return chunks

In [10]:
def get_data_from_uncomtrade(reporter,partner,flowCode,frequency,data_date,hs_code,hs_2_digit=None,hs_4_digit=None,hs_6_digit=None,include_none=False):

    """
    Fetch trade data from UN Comtrade based on specified criteria.

    Parameters:
    - reporter (str or list): Reporting country or countries.
    - partner (str or list): Partner country or countries.
    - frequency (str): Data frequency (e.g., 'annual', 'monthly').
    - data_date (str): Data period(s) in a comma-separated string.
    - hs_code (list or str): HS codes to retrieve data for, or "00" for all 2-digit codes.
    - hs_2_digit, hs_4_digit, hs_6_digit (bool): Flags to include specific HS code levels.
    - include_none (bool): If False, excludes rows with None values in the output.

    Returns:
    - final_dataframe (DataFrame): Raw trade data from the API.
    - test_dataframe (DataFrame): Processed trade data with comparisons.
    """

    if isinstance(data_date, str):
        years_list = data_date.split(',')
        # Check the length of the list and truncate if necessary
        if len(years_list) > 12:
            print("Maximum 12 date periods can be used in date string per call")
            return None,None

    if isinstance(reporter, list):
        string_list = [str(item) for item in reporter]
        reporter = ','.join(map(str, reporter))
    elif isinstance(reporter, str):
        pass
    else:
        print("The reporter codes should be a list or comma seperated string.")
        return None

    if isinstance(partner, list):
        string_list = [str(item) for item in partner]
        partner = ','.join(map(str, partner))
    elif isinstance(partner, str):
        pass
    else:
        print("The partner codes should be a list or comma seperated string.")
        return None

    if isinstance(hs_code, list) or hs_code=="00":
        pass
    else:
        print(f"""HS code must be passed as a list of strings or should be "00" """)
        return None
    if not hs_code:
        result = True
    else:
        # Get the length of the first element
        first_length = len(hs_code[0])

        # Check if all elements have the same length
        result = all(len(item) == first_length for item in hs_code)

    if result==False:
        print("The hs code string should include only 2,4 or 6 digitis as list do not send mixed digits hs code.")
        return None
    ############################################################
    if hs_code=="00":
        hs_list_2_digit=[]
        for item in hs_data["results"]:
            if item["aggrLevel"]==2:
                hs_list_2_digit.append(item['id'])
        codes_string = ','.join(map(str, hs_list_2_digit))
        final_dataframe = comtradeapicall.getFinalData(subscription_key, typeCode='C', freqCode=frequency, clCode='HS', period=data_date,
                                        reporterCode=reporter, cmdCode=codes_string, flowCode=flowCode, partnerCode=partner,
                                        partner2Code=None, customsCode=None, motCode=None, maxRecords=250000,
                                        format_output='JSON', aggregateBy=None, breakdownMode='classic',
                                        countOnly=None, includeDesc=True)
    ############################################################
    else:
        string_list = [str(item) for item in hs_code]
        if len(string_list[0])==2:

            codes_string_2_digit = ','.join(map(str, hs_code))
            hs_list_4_digit=[]
            for item in hs_data["results"]:
                if item["aggrLevel"]==4 and item["parent"] in hs_code:
                    hs_list_4_digit.append(item['id'])
            codes_string_4_digit = ','.join(map(str, hs_list_4_digit))
            hs_list_6_digit=[]
            for item in hs_data["results"]:
                if item["aggrLevel"]==6 and item["parent"] in hs_list_4_digit:
                    hs_list_6_digit.append(item['id'])
            codes_string_6_digit = ','.join(map(str, hs_list_6_digit))


        elif len(string_list[0])==4:
            for item in hs_code:
                unique_first_two_digits = set()
                # Extract the first two digits
                first_two_digits = item[:2]
                # Add the first two digits to the set
                unique_first_two_digits.add(first_two_digits)
            # Convert the set to a list if needed
            unique_first_two_digits_list = list(unique_first_two_digits)
            codes_string_2_digit = ','.join(map(str, unique_first_two_digits_list))
            codes_string_4_digit=','.join(map(str, hs_code))
            hs_list_6_digit=[]
            for item in hs_data["results"]:
                if item["aggrLevel"]==6 and item["parent"] in hs_code:
                    hs_list_6_digit.append(item['id'])
            codes_string_6_digit = ','.join(map(str, hs_list_6_digit))

        elif len(string_list[0])==6:
            for item in hs_code:
                unique_first_four_digits = set()
                # Extract the first two digits
                first_four_digits = item[:4]
                # Add the first two digits to the set
                unique_first_four_digits.add(first_four_digits)
            # Convert the set to a list if needed
            unique_first_four_digits_list = list(unique_first_four_digits)
            codes_string_4_digit = ','.join(map(str, unique_first_four_digits_list))
            for item in unique_first_four_digits_list:
                unique_first_two_digits = set()
                # Extract the first two digits
                first_two_digits = item[:2]
                # Add the first two digits to the set
                unique_first_two_digits.add(first_two_digits)
            # Convert the set to a list if needed
            unique_first_two_digits_list = list(unique_first_two_digits)
            codes_string_2_digit = ','.join(map(str, unique_first_two_digits_list))
            codes_string_6_digit=','.join(map(str, hs_code))

        else:
            print("The hs code string should include only 2,4 or 6 digitis as list do not send mixed digits hs code.")
            return None
        if(hs_2_digit==False and hs_4_digit==False and hs_6_digit==False):
            print("At least one of the digit selections must be True")
            return None
        codes_string=""
        if(hs_2_digit==True):
            codes_string+=codes_string_2_digit+","
        if(hs_4_digit==True):
            codes_string+=codes_string_4_digit+","
        if(hs_6_digit==True):
            codes_string+=codes_string_6_digit
        codes_string = codes_string.rstrip(',')
        print(f"""Final codes_string= {codes_string} """)
        chunk_size = 100
        chunks = split_numbers(codes_string, chunk_size)
        final_dataframe=pd.DataFrame()
        for chunk in range(0,len(chunks),1):
            string_numbers = list(map(str, chunks[chunk]))
            codes_string = ','.join(map(str, string_numbers))
            mydf = comtradeapicall.getFinalData(subscription_key, typeCode='C', freqCode=frequency, clCode='HS', period=data_date,
                                        reporterCode=reporter, cmdCode=codes_string, flowCode=flowCode, partnerCode=partner,
                                        partner2Code=None, customsCode=None, motCode=None, maxRecords=250000,
                                        format_output='JSON', aggregateBy=None, breakdownMode='classic',
                                        countOnly=None, includeDesc=True)
            final_dataframe=pd.concat([final_dataframe,mydf])
        final_dataframe=final_dataframe.reset_index(drop=True)
        return final_dataframe
    ##################################################################### Added for non equal price  check
    columns = ['cifvalue', 'fobvalue', 'primaryValue']
    # Initialize the result to True

    # Initialize an empty DataFrame to store rows with differences
    rows_with_differences = []

    # Loop through each row in the final_dataframe
    for _, row in final_dataframe.iterrows():
        # Extract the values for the specified columns
        values = [row[col] for col in columns]

        # Filter out NaN values and zeros
        non_nan_values = [value for value in values if pd.notna(value) and value != 0.0]

        # Check if all non-NaN values are the same
        if len(set(non_nan_values)) > 1:
            # If there are differences, add the row to the list
            rows_with_differences.append(row)

    # Concatenate all rows with differences into a DataFrame
    failures_dataframe = pd.DataFrame(rows_with_differences)

    # Check if any differences were found and output the result
    if not failures_dataframe.empty:
        print("There are differences in the price columns")

    ####################################################################
    test_dataframe=pd.DataFrame()
    periods_list = final_dataframe['period'].unique().tolist()
    for periods in periods_list:
        filtered_df=final_dataframe[final_dataframe["period"]==periods]

        reporter_numbers = filtered_df['reporterCode'].unique().tolist()
        for reporter_loop in reporter_numbers:
            unique_numbers = filtered_df[filtered_df["reporterCode"]==reporter_loop]['cmdCode'].unique().tolist()
            for hs_codes in unique_numbers:
                mydf=filtered_df[(filtered_df["cmdCode"]==hs_codes)&(filtered_df["reporterCode"]==reporter_loop)]
                unique_partners = mydf['partnerCode'].unique().tolist()
                if len(unique_partners)>1:
                    if unique_partners[0] == 0:
                        unique_partners = unique_partners[1:]

                for partners in unique_partners:
                    mydf=filtered_df[(filtered_df["cmdCode"]==hs_codes)&(filtered_df["reporterCode"]==reporter_loop)&(filtered_df["partnerCode"]==partners)]
                    # Check if mydf is empty
                    if mydf.empty:
                        import_quantity = None
                        export_quantity = None
                        import_amount= None
                        export_amount= None
                        unit_type = None
                        difference = None
                        balance_status = None
                        Hs_description=None
                        country_tag=None
                        reporter=None
                        partner=None
                        difference_amount = None
                        balance_status_dollars = None
                        total_parent=None
                        period_data=None
                    else:
                        flow_list=mydf["flowCode"].unique().tolist()
                        if "X" not in flow_list or "M" not in flow_list:
                            continue

                        try:
                            if mydf[mydf['flowCode'] == 'M']['qty'].values[0]!=0 and mydf[mydf['flowCode'] == 'X']['qty'].values[0]!=0 :
                                import_quantity = mydf[mydf['flowCode'] == 'M']['qty'].values[0]
                                export_quantity = mydf[mydf['flowCode'] == 'X']['qty'].values[0]
                                unit_type = mydf[mydf['flowCode'] == 'X']['qtyUnitAbbr'].values[0] if not mydf[mydf['flowCode'] == 'X']['qtyUnitAbbr'].empty else None
                            else:
                                import_quantity = mydf[mydf['flowCode'] == 'M']['altQty'].values[0] if not mydf[mydf['flowCode'] == 'M']['altQty'].empty else None
                                export_quantity = mydf[mydf['flowCode'] == 'X']['altQty'].values[0] if not mydf[mydf['flowCode'] == 'X']['altQty'].empty else None
                                unit_type = mydf[mydf['flowCode'] == 'X']['altQtyUnitAbbr'].values[0] if not mydf[mydf['flowCode'] == 'X']['altQtyUnitAbbr'].empty else None
                            ################################################ Under this part new value selector selects any not NaN
                            columns = ['cifvalue', 'fobvalue', 'primaryValue']
                            # Initialize variables
                            import_amount = None
                            export_amount = None
                            # Check for the first non-NaN, non-0.0 value for 'M'
                            for col in columns:
                                values = mydf[mydf['flowCode'] == 'M'][col].values
                                if len(values) > 0 and pd.notna(values[0]) and values[0] != 0.0:
                                    import_amount = values[0]
                                    break
                            # Check for the first non-NaN, non-0.0 value for 'X'
                            for col in columns:
                                values = mydf[mydf['flowCode'] == 'X'][col].values
                                if len(values) > 0 and pd.notna(values[0]) and values[0] != 0.0:
                                    export_amount = values[0]
                                    break
                            # If either import_amount or export_amount is None, set both to None
                            if import_amount is None or export_amount is None:
                                import_amount = None
                                export_amount = None

                        except IndexError as e:
                            print("IndexError occurred:", e)
                            print("DataFrame mydf:")
                            return final_dataframe,mydf


                        Hs_description=mydf[mydf['flowCode'] == 'X']['cmdDesc'].values[0] if not mydf[mydf['flowCode'] == 'X']['cmdDesc'].empty else None
                        country_tag = mydf[mydf['flowCode'] == 'X']['reporterISO'].values[0]
                        reporter = mydf[mydf['flowCode'] == 'X']['reporterCode'].values[0]
                        partner_tag= mydf[mydf['flowCode'] == 'X']['partnerDesc'].values[0]
                        partner=mydf[mydf['flowCode'] == 'X']['partnerCode'].values[0]
                        # Calculate the differences and balance status
                        if export_quantity is not None and import_quantity is not None:
                            difference = export_quantity - import_quantity
                            balance_status = "Positive" if difference > 0 else "Negative"
                        else:
                            difference = None
                            balance_status = None

                        if export_amount is not None and import_amount is not None:
                            difference_amount = export_amount - import_amount
                            balance_status_dollars = "Positive" if difference_amount > 0 else "Negative"
                        else:
                            difference_amount = None
                            balance_status_dollars = None


                    if include_none==False:
                        if difference==None and difference_amount==None:
                            continue

                    total_parent=None
                    if len(hs_codes)>2:
                        total_parent=hs_codes[:2]
                    data = pd.DataFrame([{
                    "Period": periods,
                    "HS Code": hs_codes,
                    "Parent": total_parent,
                    "Reporter Code": reporter,
                    "Reporter": country_tag,
                    "Partner Code": partner,
                    "Partner": partner_tag,
                    "HS_Description": Hs_description,
                    "Unit Type": unit_type,
                    "Import Quantitiy": import_quantity,
                    "Export Quantitiy": export_quantity,
                    "Import/Export Diff Quantity": difference,
                    "Import/Export Balance Quantity": balance_status,
                    "Import Amount Dollars": import_amount,
                    "Export Amount Dollars": export_amount,
                    "Import/Export Diff Dollars": difference_amount,
                    "Import/Export Balance Dollars": balance_status_dollars
                }])
                    test_dataframe = pd.concat([test_dataframe, data], ignore_index=True)
    if len(failures_dataframe)>0:
        return final_dataframe,test_dataframe
    else:
        return final_dataframe,test_dataframe

Get all the HS codes data reportered


In [11]:
uncomtrade_data_hs_2,balanced_dataframe_hs_2=get_data_from_uncomtrade(reporter="484",partner=codes, flowCode ="X",frequency="A",data_date="2022",hs_code="00",hs_2_digit=True,hs_4_digit=False,hs_6_digit=False)

In [12]:
positive_top_5_items=uncomtrade_data_hs_2["cmdCode"].unique().tolist()

This calls all the exports/imports data of all countries to a specific country, the values returned are close to the onesin Atlas Dataset and BACI Datasets. All you need to change is the partners code and year

In [13]:
usa_imports_2022=get_data_from_uncomtrade(reporter="842",partner=codes,data_date="2022",flowCode='M', frequency="A", hs_code=positive_top_5_items,hs_2_digit=True,hs_4_digit=False,hs_6_digit=False)

Final codes_string= 01,02,03,04,05,06,07,08,09,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,99 


In [14]:
usa_imports_2022.head()

,typeCode,freqCode,refPeriodId,refYear,refMonth,period,reporterCode,reporterISO,reporterDesc,flowCode,flowDesc,partnerCode,partnerISO,partnerDesc,partner2Code,partner2ISO,partner2Desc,classificationCode,classificationSearchCode,isOriginalClassification,cmdCode,cmdDesc,aggrLevel,isLeaf,customsCode,customsDesc,mosCode,motCode,motDesc,qtyUnitCode,qtyUnitAbbr,qty,isQtyEstimated,altQtyUnitCode,altQtyUnitAbbr,altQty,isAltQtyEstimated,netWgt,isNetWgtEstimated,grossWgt,isGrossWgtEstimated,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate
0,C,A,20220101,2022,52,2022,842,USA,USA,M,Import,4,AFG,Afghanistan,0,W00,World,H6,HS,True,73,Iron or steel articles,2,False,C00,TOTAL CPC,0,0,TOTAL MOT,-1,N/A,0.0,False,-1,N/A,0.0,False,0.0,False,0.0,False,60633.0,55865.0,60633.0,0,False,True
1,C,A,20220101,2022,52,2022,842,USA,USA,M,Import,8,ALB,Albania,0,W00,World,H6,HS,True,73,Iron or steel articles,2,False,C00,TOTAL CPC,0,0,TOTAL MOT,-1,N/A,0.0,False,-1,N/A,0.0,False,0.0,False,0.0,False,327589.0,301623.0,327589.0,0,False,True
2,C,A,20220101,2022,52,2022,842,USA,USA,M,Import,24,AGO,Angola,0,W00,World,H6,HS,True,73,Iron or steel articles,2,False,C00,TOTAL CPC,0,0,TOTAL MOT,-1,N/A,0.0,False,-1,N/A,0.0,False,0.0,False,0.0,False,8802752.0,7859034.0,8802752.0,0,False,True
3,C,A,20220101,2022,52,2022,842,USA,USA,M,Import,31,AZE,Azerbaijan,0,W00,World,H6,HS,True,73,Iron or steel articles,2,False,C00,TOTAL CPC,0,0,TOTAL MOT,-1,N/A,0.0,False,-1,N/A,0.0,False,0.0,False,0.0,False,305482.0,305479.0,305482.0,0,False,True
4,C,A,20220101,2022,52,2022,842,USA,USA,M,Import,32,ARG,Argentina,0,W00,World,H6,HS,True,73,Iron or steel articles,2,False,C00,TOTAL CPC,0,0,TOTAL MOT,-1,N/A,0.0,False,-1,N/A,0.0,False,0.0,True,0.0,False,211789985.0,199867397.0,211789985.0,4,False,True


In [20]:
# filter out the most important variables
usa_imports_2022= usa_imports_2022[['refYear','reporterDesc','partnerDesc','partnerISO','cmdCode', 'fobvalue']]

In [21]:
# calculate the total import value
total_imports_2022 = usa_imports_2022['fobvalue'].sum()
print(f"Total import value for the year 2022: {total_imports_2022}")

Total import value for the year 2022: 3243131340182.0


In [22]:
# Group by 'partnerDesc' and sum import values
partner_export_2022 = usa_imports_2022.groupby(['partnerDesc', 'partnerISO'])['fobvalue'].sum().reset_index()

In [29]:
partner_export_2022

,partnerDesc,partnerISO,fobvalue
0,Afghanistan,AFG,2.254660e+07
1,Albania,ALB,1.449910e+08
2,Algeria,DZA,3.060778e+09
3,Andorra,AND,9.778800e+06
4,Angola,AGO,1.611020e+09
...,...,...,...
215,Viet Nam,VNM,1.275212e+11
216,Wallis and Futuna Isds,WLF,1.079010e+05
217,Yemen,YEM,3.141011e+07
218,Zambia,ZMB,1.352544e+08


In [30]:
# prints the number of countries exporting to usa
partner_export_2022.shape[0]

220

In [23]:
# export index
index_df = pd.read_csv('/content/drive/MyDrive/Trade project data/usa_data/d7f111f0-1329-4004-a6dc-a355f2385ec2_Data.csv')

In [24]:
# refine the dataframe
index_df.rename(columns={'Country Name': 'partnerDesc', 'Country Code': 'partnerISO', '2022 [YR2022]': 'index'}, inplace=True)

In [25]:
index_df= index_df[['partnerDesc', 'partnerISO', 'index']]

In [32]:
constant_df = pd.merge(partner_export_2022, index_df, on=['partnerISO','partnerDesc'], how='left')

In [34]:
constant_df['index'] = constant_df['index'].replace('..', '100')

In [35]:
constant_df['index'] = pd.to_numeric(constant_df['index'], errors='coerce')
constant_df['constant_usd'] = constant_df['fobvalue'] / (constant_df['index']) * 100

In [38]:
constant_df.head()

,partnerDesc,partnerISO,fobvalue,index,constant_usd
0,Afghanistan,AFG,2.254660e+07,143.2,1.574483e+07
1,Albania,ALB,1.449910e+08,131.2,1.105115e+08
2,Algeria,DZA,3.060778e+09,203.1,1.507030e+09
3,Andorra,AND,9.778800e+06,116.4,8.401031e+06
4,Angola,AGO,1.611020e+09,220.4,7.309527e+08


In [39]:
# calculate the total import value
total_imports_2022 = constant_df['constant_usd'].sum()
print(f"Total adjusted import value for the year 2022: {total_imports_2022}")

Total adjusted import value for the year 2022: 2380100036881.688


# services data

In [36]:
services_df = pd.read_csv('/content/drive/MyDrive/Trade project data/usa_data/Commercial Services 7.31.2024.csv')

In [40]:
# slice dataframe
usa_services = services_df[services_df['REPORTER CODE'] == 'US']
sorted_usa_services = usa_services[usa_services['FLOW'] == 'Imports'].sort_values('FLOW')

In [41]:
# sort by specif year
services_df_2022 = sorted_usa_services[sorted_usa_services.YEAR == 2022]

In [42]:
services_df_2022.head()

,FLOW,INDICATOR_CODE,REPORTER CODE,PARTNER CODE,REPORTER_COUNTRY_CODE,PARTNER_COUNTRY_CODE,REPORTING ECONOMY,PARTNER ECONOMY,SECTOR,YEAR,VALUE,UNIT
42219,Imports,SOX1,US,WL,RC840,SA000,United States of America,World,Other commercial services,2022,406552.0,Million US dollar
34676,Imports,S,US,WL,RC840,SA000,United States of America,World,Memo item: Total services,2022,713886.0,Million US dollar
49747,Imports,SC,US,WL,RC840,SA000,United States of America,World,Transport,2022,157711.0,Million US dollar
26964,Imports,SPX1,US,WL,RC840,SA000,United States of America,World,Memo item: Other services,2022,432099.0,Million US dollar
7287,Imports,SOX,US,WL,RC840,SA000,United States of America,World,Commercial services,2022,688339.0,Million US dollar


From the above dataframe get the Total services from sector column